In [1]:
import numpy as np
import pandas as pd
from astropy.io import fits
import glob
from collections import defaultdict
import sqlite3 as lite

# Read in the field information
#root = "/home/esmee/Documents/DBDM/DDM2017-master/FinalProject/" 
root = "/disks/strw9/stoop/DBDM/DDM2017-master/FinalProject/Final-Project-DBDM/"

# FieldTable

| ID | FieldID | Filename | Filter | Exp time | MJD |


In [4]:
#Read the field info
field_info = pd.read_csv(root + 'file_info_for_problem.csv')

# Make the filters of the same form as for StarTable
for field in range(1,4):
    #Find the index of filter Ks
    index_ks = np.where( (field_info['FieldID'] == field) & (field_info['Filter'] == 'Ks') )[0]
    sort = np.argsort(field_info['MJD'][index_ks])
    
    #Change to Ks-E00..
    tel = 1
    for i in index_ks[sort]:
        field_info['Filter'][i] = field_info['Filter'][i] + '_E00'+str(tel)
        tel +=1

/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [5]:
field_info

,ID,FieldID,Filename,Filter,MJD,Airmass,Exptime
0,1,1,Z-ADP.2017-01-18T11:58:36.905.fits,Z,57267.167107,1.6405,40.0
1,2,1,J-ADP.2017-01-18T11:58:35.781.fits,J,57257.050432,1.0105,48.0
2,3,1,H-ADP.2017-01-18T11:58:35.780.fits,H,57257.044108,1.0060,16.0
3,4,1,Ks-ADP.2016-05-25T15:33:39.546.fits,Ks_E002,56788.346937,1.0230,16.0
4,5,1,Ks-ADP.2017-01-18T11:58:39.907.fits,Ks_E001,56561.002016,1.0790,16.0
5,6,1,Ks-ADP.2016-05-25T15:33:43.377.fits,Ks_E003,56829.039051,1.6085,16.0
6,7,1,Y-ADP.2017-01-18T11:58:36.901.fits,Y,57267.159665,1.5605,40.0
7,8,2,Z-ADP.2017-01-18T11:58:36.905b.fits,Z,57268.167107,1.6405,40.0
8,9,2,J-ADP.2017-01-18T11:58:35.781b.fits,J,57258.050432,1.0105,48.0
9,10,2,H-ADP.2017-01-18T11:58:35.780b.fits,H,57258.044108,1.0060,16.0


# StarTable
| ID  | StarID  | FieldID | Ra  | Dec  |

# IntensityTable
| ID | StarID | Filter | Flux1 | dFlux1 | Flux2 | dFlux2 | Flux3 | dFlux3 | Mag1 | dMag1 | Mag2 | dMag2 | Mag3 | dMag3 |

In [113]:
filenames = glob.glob(root + '*.fits')
starIDs = []
#Find the unique StarIDs
for i, f in enumerate(filenames):
    hdu = fits.open(f)
    [starIDs.append(ID) for ID in hdu[1].data['StarID']]

uniq_IDs = np.unique(starIDs)
nstars = len(uniq_IDs)

#Make the Star and Intensity table
scol = np.array(['ID', 'FieldID', 'StarID', 'Ra', 'Dec'])
icol = np.array(['ID', 'StarID', 'FieldID', 'Filter', 'Flux1', 'dFlux1', 'Flux2', 'dFlux2', 'Flux3', 'dFlux3', 'Mag1', 'dMag1', 'Mag2', 'dMag2', 'Mag3', 'dMag3'])
StarTable = pd.DataFrame(data = np.zeros((nstars, len(scol))), columns = scol)
IntensityTable = pd.DataFrame(data = np.zeros((len(starIDs), len(icol))), columns = icol)

In [142]:
# Fill the tables
seen = []
first = False
for fname in filenames:
    
    print "Started with file {0}".format(fname.replace(root, ''))
    
    hdu = fits.open(fname)
    fname = fname.replace(root, '')
    sID = hdu[1].data['StarID'] #StarIDs
    fID = fname[6] #FieldID
    filt = fname[8:-5] # filter 
    if '-' in filt: filt = filt.replace('-','_')
    
    #Index in table of the stars
    sindex = [np.where(uniq_IDs == i)[0][0] for i in sID]
    
    # Is this field already seen
    if not sID[0] in seen:
        
        # Fill ID, FieldID 
        StarTable['ID'][sindex] = sindex
        StarTable['FieldID'][sindex] = fID
        # Fill StarID, Ra and Dec
        for c in scol[2:5]:
            StarTable[c][sindex] = hdu[1].data[c]
        
        [seen.append(ID) for ID in sID]
    
    if first == False:
        iindex = np.arange(len(sID))
        first = True
    else:
        iindex += len(sID)
    
    print filt, iindex[-1]
    
    # Fill the Intensity Table
    IntensityTable['ID'][iindex] = iindex
    IntensityTable['StarID'][iindex] = sID
    IntensityTable['Filter'][iindex] = filt
    
    IntensityTable['FieldID'][iindex] = fID
    # Fill in the flux and magnitudes in IntensityTable
    for c in icol[4:-1]:
        IntensityTable[c][iindex] = hdu[1].data[c]
    


Started with file Field-3-Ks-E002.fits


/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Ks_E002 9999


/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a sl

Started with file Field-3-Ks-E001.fits
Ks_E001 19999
Started with file Field-2-H.fits
H 29999
Started with file Field-1-Ks-E001.fits
Ks_E001 39999
Started with file Field-2-Y.fits
Y 49999
Started with file Field-3-J.fits
J 59999
Started with file Field-1-Y.fits
Y 69999
Started with file Field-1-Ks-E003.fits
Ks_E003 79999
Started with file Field-1-Ks-E002.fits
Ks_E002 89999
Started with file Field-1-H.fits
H 99999
Started with file Field-3-H.fits
H 109999
Started with file Field-3-Z.fits
Z 119999
Started with file Field-2-Z.fits
Z 129999
Started with file Field-3-Y.fits
Y 139999
Started with file Field-1-Z.fits
Z 149999
Started with file Field-2-Ks-E001.fits
Ks_E001 159999
Started with file Field-2-J.fits
J 169999
Started with file Field-1-J.fits
J 179999


/software/local/lib64/python2.7/site-packages/ipykernel/__main__.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [143]:
StarTable

,ID,FieldID,StarID,Ra,Dec
0,0.0,1,0.0,4.706625,-0.529796
1,1.0,1,1.0,4.706900,-0.529931
2,2.0,1,2.0,4.706090,-0.529531
3,3.0,1,3.0,4.705330,-0.529153
4,4.0,1,4.0,4.705183,-0.529083
5,5.0,1,5.0,4.706955,-0.529958
6,6.0,1,6.0,4.706450,-0.529710
7,7.0,1,7.0,4.706465,-0.529715
8,8.0,1,8.0,4.704572,-0.528783
9,9.0,1,9.0,4.705731,-0.529352


In [124]:
# write tables to csv
StarTable.to_csv(root+'StarTable.csv')
field_info.to_csv(root+'FieldTable.csv')
IntensityTable.to_csv(root + 'IntensityTable.csv')

In [127]:
# Read the Tables
FieldTable = pd.read_csv(root+ "FieldTable.csv")
StarTable = pd.read_csv(root+ "StarTable.csv")
IntensityTable = pd.read_csv(root + 'IntensityTable.csv')

FieldTable.drop('Unnamed: 0', axis=1, inplace=True)
StarTable.drop('Unnamed: 0', axis=1, inplace=True)
IntensityTable.drop('Unnamed: 0', axis=1, inplace=True)

# Make the database
con = lite.connect(root+"VVV.db")
StarTable.to_sql('StarTable', con, if_exists = 'replace')
FieldTable.to_sql('FieldTable', con, if_exists = 'replace')
IntensityTable.to_sql('IntensityTable', con, if_exists = 'replace')

# Perform the queries

**R1:** Find all images observed between MJD = 56800 and MJD = 57300 and give me the number of stars detected with S/N > 5 in each image.

In [128]:
con = lite.connect(root + 'VVV.db')

command1 = """SELECT f.Filename, COUNT(i.StarID) FROM FieldTable AS f JOIN IntensityTable AS i ON f.FieldID = i.FieldID AND f.Filter = i.Filter \
            WHERE (i.Flux1/i.dFlux1 > 5) AND (f.MJD BETWEEN 56800 AND 57300) \
            GROUP BY f.Filename"""
t = pd.read_sql(command1, con)

print "The images taken between 56800 and 57300 and the number of stars with S/N > 5 "
print t

The images taken between 56800 and 57300 and the number of stars with S/N > 5 
                               Filename  COUNT(i.StarID)
0    H-ADP.2017-01-18T11:58:35.780.fits             9990
1   H-ADP.2017-01-18T11:58:35.780b.fits             9985
2   H-ADP.2017-01-18T11:58:35.780c.fits             9987
3    J-ADP.2017-01-18T11:58:35.781.fits             9985
4   J-ADP.2017-01-18T11:58:35.781b.fits             9985
5   J-ADP.2017-01-18T11:58:35.781c.fits             9983
6   Ks-ADP.2016-05-25T15:33:43.377.fits             9992
7    Y-ADP.2017-01-18T11:58:36.901.fits             9967
8   Y-ADP.2017-01-18T11:58:36.901b.fits             9985
9   Y-ADP.2017-01-18T11:58:36.901c.fits             9977
10   Z-ADP.2017-01-18T11:58:36.905.fits             9862
11  Z-ADP.2017-01-18T11:58:36.905b.fits             9968
12  Z-ADP.2017-01-18T11:58:36.905c.fits             9930


**R2:** Find the objects that have J-H > 1.5

In [140]:
command2 = """ SELECT i1.StarID FROM IntensityTable AS i1 \
                JOIN IntensityTable AS i2 ON i1.StarID = i2.StarID \
                WHERE (i1.Filter = 'J') AND (i2.Filter = 'H') \
                AND (i1.Flux1 - i2.Flux1 + i1.dFlux1 + i2.dFlux1 > 1.5) \
                AND (i1.Flux1 - i2.Flux1 - i1.dFlux1 - i2.dFlux1 > 1.5)"""

t = pd.read_sql(command2, con)

print "Objects with J-H > 1.5 are the following stars:"
print t


Objects with J-H > 1.5 are the following stars:
        StarID
0     300022.0
1     300054.0
2     300057.0
3     300096.0
4     300153.0
5     300158.0
6     300190.0
7     300208.0
8     300220.0
9     300240.0
10    300331.0
11    300354.0
12    300361.0
13    300363.0
14    300559.0
15    300593.0
16    300594.0
17    300595.0
18    300623.0
19    300642.0
20    300647.0
21    300754.0
22    300757.0
23    300782.0
24    300791.0
25    300823.0
26    300858.0
27    300861.0
28    300872.0
29    300873.0
...        ...
1198    8971.0
1199    9066.0
1200    9073.0
1201    9087.0
1202    9109.0
1203    9146.0
1204    9152.0
1205    9203.0
1206    9229.0
1207    9231.0
1208    9236.0
1209    9288.0
1210    9304.0
1211    9305.0
1212    9341.0
1213    9356.0
1214    9446.0
1215    9455.0
1216    9596.0
1217    9601.0
1218    9636.0
1219    9693.0
1220    9726.0
1221    9852.0
1222    9886.0
1223    9917.0
1224    9923.0
1225    9961.0
1226    9965.0
1227    9999.0

[1228 rows x 1 column

(array([], dtype=int64),)